In [48]:
import random
from tkinter import N
import numpy as np
from itertools import groupby

def problem(N, seed=None):
    random.seed(seed)
    return [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]

def fitness(individual):
    goal = set(x for x in range(N))
    unique_numbers = set()
    for gene in individual:
        for number in gene:
            unique_numbers.add(number)
    missing_numbers = goal - unique_numbers
    all_elements = [single_element for l in individual for single_element in l]
    duplicates = sum([len(list(group))-1 for key, group in groupby(sorted(all_elements))])
    return -1000*len(missing_numbers) - duplicates

def init(mu):
    population_list = []
    for _ in range(mu):
        individual = random.choices(STATE_SPACE, k=random.randint(1, len(STATE_SPACE)))
        fitness_value = fitness(individual)
        population_list.append((fitness_value, individual))
    return population_list

def wheel(population):
    population.sort(reverse=True)
    parent = []
    while len(parent) <= 1:
        pick = random.randint(0, mu*(mu+1)/2)
        inc = mu - 1
        parent_index = 0
        curr = mu
        while pick > curr:
            parent_index += 1
            curr += inc
            inc -= 1
        parent = population[parent_index][1]
    return population[parent_index]

def crossover(population):
    offspring = []        
    for _ in range(round(offspring_size/2)):
        parent1, parent2 = wheel(population), wheel(population)
        if len(parent1[1]) <= 1 or len(parent2[1]) <= 1:
            print("Parents are too short: ", parent1, parent2)
            input()
        cut1 = random.randint(1, len(parent1[1])-1)
        cut2 = random.randint(1, len(parent2[1])-1)
        #print("Cut 1 and 2 are", cut1, cut2)
        child1 = parent1[1][:cut1] + parent2[1][cut2:]
        child2 = parent1[1][cut1:] + parent2[1][:cut2]
        childs = [child1, child2]

        for child in childs:
            if random.random() < epsilon: #Probability to mutate the child
                child = mutation(child)
            fitness_value = fitness(child)
            offspring = offspring + [(fitness_value, child)]
    return offspring
        

def mutation(individual):
    individual.pop(random.randrange(len(individual)))
    individual.append(random.choice(STATE_SPACE))
    return individual        


In [49]:
N = 20
seed = 42
STATE_SPACE = problem(N, seed)
seed = None


In [50]:
#Initialize population and store as tuples in the population list
mu = 30
offspring_size = 20
epsilon = 0.1 # Mutation probability 
population = init(mu)
best_solution = population[0] #Pick a random initial best solution
steady_state_count = 0

for _ in range(2*500):
    #print('Crossover the', _+1, '-th time.')
    population = population + crossover(population)
    population.sort(reverse=True)
    if population[0][0] > best_solution[0]:
        best_solution = population[0]
        steady_state_count = 0
    population = population[:mu]
    steady_state_count += 1
    if steady_state_count == 20: 
        for i in range(len(population)):
            individual = population.pop(0)
            mutated_individual = mutation(individual[1])
            population.append((fitness(mutated_individual), mutated_individual))
            

all_elements = [single_element for l in best_solution[1] for single_element in l]
duplicates = sum([len(list(group))-1 for key, group in groupby(sorted(all_elements))])
print(duplicates)

goal = set(x for x in range(N))
unique_numbers = set()
for gene in best_solution[1]:
    for number in gene:
        unique_numbers.add(number)
missing_numbers = goal - unique_numbers
print(missing_numbers)
print(best_solution)

9
{9, 11, 12}
(-6, [[3, 6, 7, 10, 14, 17], [0, 1, 2, 7], [4, 5, 8, 13, 15, 16, 17, 19], [0, 1, 2, 3, 6, 13, 17, 18]])
